In [1]:
# import modules

import numpy as np
import pandas as pd
import sys
from scipy.linalg import sqrtm
from sklearn.decomposition import TruncatedSVD
from hybridCorr import matcor as hbcr

In [4]:
# define time-lagged cannonical correlation analysis

def TCCA(time_series,lag_time,periodicity=[0,0]):
        """
        ----------------- Time-lagged Independent Component Analysis -----------------
        Return:
        Eigenvalue and Eigenvector matrices of the diagonalized time-lagged cannonical
        correlation matrix
        ------------------------------------------------------------------------------
        """
        # Create time-lagged dataset
        tau1 = time_series[:-lag_time,:]
        tau2 = time_series[lag_time:,:]
        # Calculate time-lagged hybrid correlation matrices
        C00 = hbcr(circ=periodicity).run(tau1,tau1)
        C01 = hbcr(circ=periodicity).run(tau1,tau2)
        C11 = hbcr(circ=periodicity).run(tau2,tau2)
        # Estimate Koopman operator
        CovMat = np.matmul(np.linalg.inv(sqrtm(C00)),np.matmul(C01,np.linalg.inv(sqrtm(C11))))
        # Singular value decomposition
        lsv,sm,rsv = np.linalg.svd(CovMat)
        self.tcc_Cov = CovMat
        return lsv,sm,rsv

In [5]:
# define VAMP scoring system

def VAMP_n(time_series,lag_time,periodicity=[0,0],N=2):
        """
        ----------------- Variational Approach to Markov Process Score -----------------
        Return:
        Sum of the diagonal elements raised to the power n, of the diagonalized time-
        lagged cannonical correlation matrix
        --------------------------------------------------------------------------------
        """
        # get singular value decomposed cannonical correlation matrix
        _,Diag,_ = TCCA(time_series=time_series,lag_time=lag_time,periodicity=periodicity)
        # extract vamp-n score
        VAMP_n = np.sum(np.power(Diag,N))+1
        return VAMP_n